In [1]:
import numpy as np
import pandas as pd

In [2]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

### Подгружаем данные

In [3]:
events_data_train = pd.read_csv('data/event_data_train.csv')
submission_data_train = pd.read_csv('data/submissions_data_train.csv')

In [4]:
events_data_train

,step_id,timestamp,action,user_id
0,32815,1434340848,viewed,17632
1,32815,1434340848,passed,17632
2,32815,1434340848,discovered,17632
3,32811,1434340895,discovered,17632
4,32811,1434340895,viewed,17632
...,...,...,...,...
3480698,33375,1526772685,viewed,11677
3480699,32815,1526772750,viewed,5198
3480700,32815,1526772750,discovered,5198
3480701,32815,1526772750,passed,5198


In [5]:
submission_data_train

,step_id,timestamp,submission_status,user_id
0,31971,1434349275,correct,15853
1,31972,1434348300,correct,15853
2,31972,1478852149,wrong,15853
3,31972,1478852164,correct,15853
4,31976,1434348123,wrong,15853
...,...,...,...,...
509099,120745,1501946959,wrong,2615
509100,120745,1503059927,correct,13177
509101,120745,1506348153,wrong,15253
509102,120745,1523035316,wrong,1424


### Для user_id формируем агрегированные данные по 2м датасетам ( количество step_id, viewed, passed, discovered, started_attempt, correct и wrong попыток)

In [6]:
def counting_values(df, col_name, column_value):
    # Функция для подсчета количества 
    # (step_id, viewed, passed, discovered, started_attempt) 
    # для каждого пользователя
    
    return df[df[col_name] == column_value].groupby('user_id').agg({col_name : 'count'}).rename({col_name : 'n_' + column_value}, axis = 1)

In [7]:
def df_processing(df_events_data, df_submission_data):
    
     # Создаем X_train, с 1 колонкой user_id
    X_train = pd.DataFrame(df_events_data.drop(list(df_events_data), axis = 1).index.unique())
        
    # Устанавливаем user_id как индекс
    X_train = X_train.set_index('user_id')

    # Для каждого уникального значения колонки action df_events_data подсчитываем количество значений
    for column_value in df_events_data.action.unique():
        X_train = X_train.join( counting_values(df_events_data, 'action', column_value) )
        
    # Добавляем количество step_id для каждого юзера
    n_users_steps = df_events_data.groupby('user_id').agg({'step_id' : 'nunique'}).rename({'step_id' : 'n_steps'}, axis = 1)
    X_train = X_train.join( n_users_steps )

    # Подсчитываем количество correct и wrong ответов
    X_train = X_train.join( counting_values(df_submission_data, 'submission_status', 'correct') )
    X_train = X_train.join( counting_values(df_submission_data, 'submission_status', 'wrong') )
    
    # Заменяем NaN на нули
    X_train = X_train.fillna(0)
    
    return X_train

In [8]:
%%time
full_users_data = df_processing(events_data_train.set_index('user_id'), submission_data_train)

Wall time: 7.49 s


In [9]:
# Отсортируем по индексу
full_users_data = full_users_data.sort_index()
# Добавляем целевую колонку
full_users_data['is_completed_course'] = full_users_data.n_passed > 40

full_users_data

,n_viewed,n_passed,n_discovered,n_started_attempt,n_steps,n_correct,n_wrong,is_completed_course
user_id,,,,,,,,
1,1.0,0.0,1.0,0.0,1,0.0,0.0,False
2,10.0,9.0,9.0,2.0,9,2.0,0.0,False
3,192.0,87.0,91.0,30.0,91,29.0,23.0,True
5,12.0,11.0,11.0,4.0,11,2.0,2.0,False
7,1.0,1.0,1.0,0.0,1,0.0,0.0,False
...,...,...,...,...,...,...,...,...
26790,9.0,8.0,8.0,1.0,8,1.0,0.0,False
26793,1.0,0.0,1.0,1.0,1,0.0,0.0,False
26794,180.0,69.0,69.0,34.0,69,33.0,9.0,True


## Оставляем из 2х датасетов данные только за 3 первые дня каждого юзера

In [10]:
def three_days_df(df1, df2, n_days = 3):

    # Считаем число дней в виде timestam
    n_days_timestamp = n_days * 24 * 60 * 60
    
    # Добавляем колонку с минимальным timestamp для каждого юзера
    users_min_timestamp = df1.groupby('user_id', as_index = False).agg({'timestamp' : 'min'}).rename({'timestamp' : 'min_timestamp'}, axis = 1)
    
    # Добаляем эту фичу в наш df
    df1 = df1.set_index('user_id').join(users_min_timestamp.set_index('user_id'))
    df2 = df2.set_index('user_id').join(users_min_timestamp.set_index('user_id'))
    
    # Выбираем только те значения, где timestamp меньше, чем минимальный + 3(n) дня
    df1 =  df1[df1.timestamp < df1.min_timestamp + n_days_timestamp]
    df2 =  df2[df2.timestamp < df2.min_timestamp + n_days_timestamp]
    
    # Удаляем добавленную ранее фичу
    df1 = df1.drop('min_timestamp', axis = 1)
    df2 = df2.drop('min_timestamp', axis = 1)
    
    return df1, df2

In [11]:
ed, sd = three_days_df(events_data_train, submission_data_train)

In [12]:
ed

,step_id,timestamp,action
user_id,,,
1,31981,1472827464,viewed
1,31981,1472827464,discovered
2,32815,1514383364,discovered
2,32815,1514383364,viewed
2,32815,1514383364,passed
...,...,...,...
26798,31956,1520978499,passed
26798,31956,1520978499,viewed
26798,31955,1520978742,passed


In [13]:
sd

,step_id,timestamp,submission_status
user_id,,,
2,32812,1514383485,correct
2,32929,1514383420,correct
3,31977,1434359432,wrong
3,31977,1434359520,wrong
3,31977,1434359541,wrong
...,...,...,...
26797,32812,1508425828,correct
26797,32929,1508425367,correct
26798,32812,1520978063,wrong


### Получили 2 датасета (ed и sd) с информацией за 3 дня

## Формируем X_train и y_train

In [14]:
y_train = full_users_data.is_completed_course.map(int)
y_train

user_id
1        0
2        0
3        1
5        0
7        0
        ..
26790    0
26793    0
26794    1
26797    0
26798    0
Name: is_completed_course, Length: 19234, dtype: int64

In [15]:
%%time
X_train = df_processing(ed, sd)

Wall time: 1.57 s


In [16]:
X_train

,n_viewed,n_discovered,n_passed,n_started_attempt,n_steps,n_correct,n_wrong
user_id,,,,,,,
1,1.0,1.0,0.0,0.0,1,0.0,0.0
2,9.0,9.0,9.0,2.0,9,2.0,0.0
3,20.0,15.0,15.0,4.0,15,4.0,4.0
5,1.0,1.0,1.0,0.0,1,0.0,0.0
7,1.0,1.0,1.0,0.0,1,0.0,0.0
...,...,...,...,...,...,...,...
26790,6.0,6.0,6.0,1.0,6,1.0,0.0
26793,1.0,1.0,0.0,1.0,1,0.0,0.0
26794,134.0,61.0,61.0,32.0,61,31.0,9.0


### Сохраняем данные

In [17]:
X_train.to_csv('X_train.csv')
y_train.to_csv('y_train.csv')